In [7]:
import requests
import pandas as pd

# 取得資料
res = requests.get("https://isin.twse.com.tw/isin/C_public.jsp?strMode=2")
df = pd.read_html(res.text)[0]

# 設定 column 名稱
df.columns = df.iloc[0]
df = df.iloc[1:]

# 移除row與column，超過三個NaN則移除
df = df.dropna(thresh=3, axis=0).dropna(thresh=3, axis=1)

# 只提取「有價證券代號及名稱」這一列，並將代號和名稱分開
df[['stock_id', 'name']] = df['有價證券代號及名稱'].str.split('　', expand=True)

# 過濾出stock_id只包含數字的行
df = df[df['stock_id'].str.isdigit()]

# 保存結果為 CSV 檔案，不包含標頭
df[['stock_id', 'name']].to_csv('stock_id.csv', index=False, header=True, encoding='utf-8-sig')

print("已成功將過濾後的資料保存為 stock_id.csv")

/var/folders/yn/hmlh5xbj2_v9k4x75hntmgqw0000gn/T/ipykernel_11976/1442526003.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(res.text)[0]


已成功將過濾後的資料保存為 stock_id.csv


參考 

In [5]:
import yfinance as yf
import pandas as pd
import time

# 讀取csv檔
stock_list = pd.read_csv('stock_id.csv')
stock_list.columns = ['stock_id', 'name']
historical_data = pd.DataFrame()

for i in stock_list.index:
    # 抓取股票資料
    stock_id = str(stock_list.loc[i, 'stock_id']) + '.TW'
    data = yf.Ticker(stock_id)
    df = data.history(period="max")
    
    # 增加股票代號
    df['stock_id'] = stock_list.loc[i, 'stock_id']
    
    # 合併
    historical_data = pd.concat([historical_data, df])
    time.sleep(0.8)

historical_data.to_csv('historical_data.csv', index=False)